In [1]:
#from styx_msgs.msg import TrafficLight
import tensorflow as tf
import numpy as np
import cv2
import zipfile
import tarfile
import os
import sys
import random
from PIL import Image
import matplotlib.pyplot as plt

if not os.path.exists('object_detection'):
    print("Copy object_detection directory from $(Tensorflow)\models\research directory")
    exit()

sys.path.append('object_detection')
sys.path.append('object_detection/utils')

from collections import defaultdict

#from utils import label_map_util
#from utils import visualization_utils as vis_util

import label_map_util
import visualization_utils as vis_util

#import rospy

from urllib.request import urlopen
import six.moves.urllib as urllib

class TLClassifier(object):
    def __init__(self, download=False):
        self.MODEL_NAME = None
        self.MODEL_NAME = self.setup_ssd_model()
        print("using ssd model: " + self.MODEL_NAME)
        MODEL_PATH = 'models/' + self.MODEL_NAME
        # Path to frozen detection graph.
        PATH_TO_CKPT = MODEL_PATH + '/frozen_inference_graph.pb'

        self.model = None
        self.width = 0
        self.height = 0
        self.channels = 3
        self.gamma = 0.6
        self.image_count = 0
        self.correct_gamma = True

        # Load a frozen model into memory
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

            self.sess = tf.Session(graph=self.detection_graph)
            # Input and output Tensors for detection_graph
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')
        self.category_index = self.get_category_index()

    def get_model_name(self):
        return self.MODEL_NAME

    def setup_ssd_model(self):
        MODEL_NAME = None
        MODEL_NAME = 'alex-lechner_udacity_trained_model'
        #MODEL_NAME = 'real_mobilenets_ssd_38k_epochs'
        #MODEL_NAME = 'sim_mobilenets_ssd_30k_epochs'
        return MODEL_NAME

        if "1.3.0" in tf.__version__:
            #MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
            MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
        else:
            #MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09'
            MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
        """Downloads coco model"""
        MODEL_FILE = 'models/' + MODEL_NAME + '.tar.gz'
        DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
        if os.path.exists('models/' + MODEL_NAME):
            print("model already downloaded")
            return
        else:
            URL_PATH = DOWNLOAD_BASE + MODEL_NAME + '.tar.gz'
            print("downloading model from " + URL_PATH)
            response = urlopen(URL_PATH)
            f = open(MODEL_FILE, 'wb')
            f.write(response.read())
            f.close()
            print("downloaded model successfully")
        print("extracting model")
        tar_file = tarfile.open(MODEL_FILE)
        for file in tar_file.getmembers():
            file_name = os.path.basename(file.name)
            if 'frozen_inference_graph.pb' in file_name:
                tar_file.extract(file, os.getcwd()+ '/models')
                #tar_file.extractall(os.getcwd()+ '/models')
                print("extracted model successfully")
        tar_file.close()
        os.remove(MODEL_FILE) # remove .tar.gz file
        return MODEL_NAME

    def get_category_index(self):
        PATH_TO_LABELS = r'models\alex-lechner_udacity_trained_model\data\udacity_label_map.pbtxt'
        NUM_CLASSES = 13
        label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
        categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)
        #print(category_index)
        return category_index

    def load_image_into_numpy_array(self, image):
        (im_width, im_height) = image.size
        return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

    def get_classification(self, img_path):
        tl_index = None
        tl_color = None
        image = Image.open(img_path)
        image_np = self.load_image_into_numpy_array(image)
        image_expanded = np.expand_dims(image_np, axis=0)

        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_expanded})

        #print('SCORES')
        #print(scores[0])
        #print('CLASSES')
        #print(classes[0])

        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np, 
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            self.category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            line_thickness=8)
        #plt.figure(figsize=(12,8))
        #plt.imshow(image_np)
        #plt.show()
        return image_np, tl_index, tl_color

In [2]:
from os.path import isfile, join

print("Start")
light_classifier = TLClassifier(True)
model_name = light_classifier.get_model_name()

#pathIn= './input/rosbag_data_1/images/'
pathIn= './input/rosbag_data_2/images/'
pathOut = './models/' + model_name + '/video.avi'
fps = 5.0

frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#for sorting the file names properly
files.sort(key = lambda x: int(x[5:-4]))
print("Processing images and creating video")
for i in range(len(files)):
    filepath=pathIn + files[i]
    #reading each file
    #img = cv2.imread(filepath)
    img, color_index, tl_color = light_classifier.get_classification(filepath)
    height, width, layers = img.shape
    size = (width,height)
    #inserting the frames into an image array
    frame_array.append(img)
    #break
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()
print("Done")

Start
using ssd model: alex-lechner_udacity_trained_model
Processing images and creating video
Done
